# Project Phase 1 
## SimBoost


#### Name: kayvan shabani
#### Student No.: 95106375

Drug discovery is a time-consuming, laborious, costly and high-risk process. According to a report by the Eastern Research Group (ERG), it usually takes 10-15 years to develop a new drug. However, the success rate of developing a new molecular entity is only 2.01%. \
Finding a compound that selectively binds to a particular protein is a highly challenging and typically expensive procedure in the drug development process. \
In this project we are going to implement [SimBoost](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5395521/#CR42) which is machine-learning approch for predicting drug–target binding affinities using gradient boosting.




We will be using [Davis](http://staff.cs.utu.fi/~aatapa/data/DrugTarget/) dataset, which contains selectivity assays of the kinase protein family and the relevant inhibitors with their respective dissociation constant (Kd) values. It comprises interactions of 442 proteins and 68 drugs.

## 1. Setup

### 1.1 Imports libs

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### 1.2 Loading data

We need following files for this project:
- `target_gene_names.txt`: gene names of the targets
- `drug_PubChem_CIDs.txt`: PubChem CIDs of the drugs
- `drug-drug_similarities_2D.txt`: drug-drug structural fingerprint similarities computed the Structure Clustering sever at PubChem
- `target-target_similarities_WS_normalized.txt`: target-target sequence similarities computed with the normalized versions of the Smith-Waterman (SW) score.
- `drug-target_interaction_affinities_Kd__Davis_et_al.2011.txt`: drug-target interaction affinities.

The rows (and columns) of the drug-drug similarity matrices correspond to the rows of the interaction affinity matrix and the rows (and columns) of the target-target similarity matrices correspond to the columns of the interaction affinity matrix.

In [ ]:
target_gene_names = pd.read_csv("target_gene_names.txt", header=None, index_col = 0)
drug_pubchemIDs = pd.read_csv("drug_PubChem_CIDs.txt", header=None, index_col = 0) #  dtype=str

#################################################################################
sim_targets = pd.read_csv("target-target_similarities_WS_normalized.txt" , header=None, sep=r"\s+")

sim_targets.index = target_gene_names.index
sim_targets.columns = target_gene_names.index
sim_drugs = pd.read_csv("drug-drug_similarities_2D.txt", header=None, sep=r"\s+")
sim_drugs.index = drug_pubchemIDs.index
sim_drugs.columns = drug_pubchemIDs.index
bindings = pd.read_csv("drug-target_interaction_affinities_Kd__Davis_et_al.2011.txt", header=None, sep=r"\s+")
bindings.index = drug_pubchemIDs.index
bindings.columns = target_gene_names.index

sim_targets.shape, sim_drugs.shape, bindings.shape

### 1.3 Preprocessing

In davis dataset, standard value is Kd in nM. we used the transformation below:

### $pK_{d}=-log_{10}\frac{K_d}{1e9}$ 


In [ ]:
drug_pubchemIDs.sort_index(inplace=True)
target_gene_names.sort_index(inplace=True)

sim_targets.sort_index(inplace=True)
sim_drugs.sort_index(inplace=True)
bindings.sort_index(inplace=True)
sim_targets = sim_targets/100
#sim_drugs = None
transformed_bindings = (-1)*np.log10(bindings/1000000000)

transformed_bindings.head()

In [ ]:

x = transformed_bindings.stack()
binsnum = np.unique(x.get_values()).size
x.plot.hist(bins=binsnum)


As you can see in histogram. The peak at pKd value 5 (10000 nM), These values correspond to the negative pairs that either have very weak binding affinities (Kd > 10000nM) or are not observed in the primary screen.

### 1.4 Drug-Target-Binding

In [ ]:
products = {'Product': ['Tablet','iPhone','Laptop','Monitor'],
            'Price': [250,800,1200,300]
            }

df = pd.DataFrame(products, columns= ['Product', 'Price'])
df

In [ ]:

drug_target_binding = pd.DataFrame(transformed_bindings.stack().to_frame().set_index(0, append=True).index.tolist(), 
                                   index=transformed_bindings.stack().to_frame().set_index(0, append=True).index.tolist(),
                                   columns=['Drug', 'Target', 'Binding_Val'])

drug_target_binding.head()

In [ ]:
temp1 = pd.concat([target_gene_names]*68)

res = pd.concat([drug_target_binding.reset_index(drop=True), temp1.reset_index(drop=True)], axis=1)
res = res.sort_values(by=['Target', 'Drug'])
temp2 = pd.concat([drug_pubchemIDs]*442)
res = pd.concat([res.reset_index(drop=True), temp2.reset_index(drop=True)], axis=1)
res = res.sort_values(by=['Drug','Target'])
res.index = drug_target_binding.index
res

In [ ]:
test = pd.DataFrame({
    'id' : ['a', 'b', 'c', 'd'],
    'times' : [2, 3, 1, 5]
    })

pd.concat([test]*2)

### 1.5 Train, Validation and Test Datasets

In [ ]:


split1 = train_test_split(drug_target_binding, test_size = 0.3)
train_data = split1[0]
split2 = train_test_split(split1[1], test_size = 0.3)
val_data = split2[0]
test_data = split2[1]

train_data.head()

## 2.Feature Engineering 

In this part we are going to extract some feature for each target/drug. after that we will replace these features with their names/pubchemIDs. 

### 2.1 Average Similarities and Binding values

In [ ]:

new = train_data[['Target','Binding_Val']].groupby('Target').mean()
target_gene_names['avg-binding'] = new
target_gene_names['avg-sim'] = sim_targets.mean(axis=1)

print(target_gene_names.mean())
target_gene_names.head()


In [ ]:

new = train_data[['Drug','Binding_Val']].groupby('Drug').mean()
drug_pubchemIDs['avg-binding'] = new
drug_pubchemIDs['avg-sim'] = sim_drugs.mean(axis = 1)

drug_pubchemIDs.head()

### 2.2 Drug/Target Similarity Networks

#### 2.2.1 Build Networks

I build two networks one for drugs and another one for targets. The nodes are drugs or targets, and an edge between two nodes exists if their similarity is above a threshold. \
For building networks, we are going to use [igraph](https://igraph.org/) package. 

In [ ]:
import igraph

In [ ]:

drug_sim_threshold = 0.56
target_sim_threshold = 0.51

targetonezero = sim_targets.copy()
targetonezero.values[[np.arange(targetonezero.shape[0])]*2] = 0
targetonezero = targetonezero.where(targetonezero >= target_sim_threshold,0)
targetonezero = targetonezero.where(targetonezero < target_sim_threshold,1)

drugonezero = sim_drugs.copy()
drugonezero.values[[np.arange(drugonezero.shape[0])]*2] = 0
drugonezero = drugonezero.where(drugonezero >= drug_sim_threshold,0)
drugonezero = drugonezero.where(drugonezero < drug_sim_threshold,1)

drug_graph = igraph.Graph.Adjacency(drugonezero.values.tolist())
target_graph = igraph.Graph.Adjacency(targetonezero.values.tolist())

igraph.plot(drug_graph)


i computed average of that columns and used average for threshhold

#### 2.2.2 Number of neighbors, PageRank

In [ ]:


drug_pubchemIDs['n_neighbors'] = drugonezero.sum(axis = 1)
drug_pubchemIDs['page_rank'] = drug_graph.pagerank()

target_gene_names['n_neighbors'] = targetonezero.sum(axis = 1)
target_gene_names['page_rank'] = target_graph.pagerank()

target_gene_names.head()

### 2.3 Non-negative Matrix Factorization

For extracting features from binding affinity matrix we are going to use [Non-negative Matrix Factorization](https://en.wikipedia.org/wiki/Non-negative_matrix_factorization). 

In NMF, we are trying to approximately factor a matrix $B ∈ R_+^{d×t}$ (binding affinity matrix of d drugs and t targets) into two matrices $P ∈ R_+^{k×d}$ and $Q ∈ R_+^{k×t}$ which  $B = P^TQ$

The columns of the factor matrices P and Q are utilized as parts of the feature vectors for the drugs and targets respectively.

In [ ]:
from sklearn.decomposition import NMF

In [ ]:

latent_dim = 5
train_binding_matrix = train_data.pivot_table(columns='Target', index='Drug', values='Binding_Val')
train_binding_matrix = train_binding_matrix.fillna(5)

train_binding_matrix.head()

In [ ]:
train_binding_matrix.columns

In [ ]:

nmf = NMF(n_components=latent_dim)
P = nmf.fit_transform(train_binding_matrix);
Q = nmf.components_;
#nR = np.dot(P,Q)
#nR
drug_pubchemIDs = pd.concat([drug_pubchemIDs.reset_index(drop=True),pd.DataFrame(data=P).reset_index(drop=True)], axis=1)
drug_pubchemIDs.index = train_binding_matrix.index
drug_pubchemIDs.columns = ['avg-binding-drug', 'avg-sim-drug','n_neighbors-drug','page_rank-drug','0-drug','1-drug','2-drug','3-drug','4-drug']

target_gene_names = pd.concat([target_gene_names.reset_index(drop=True),pd.DataFrame(data=Q.T).reset_index(drop=True)], axis=1)
target_gene_names.index = train_binding_matrix.columns
target_gene_names.columns = ['avg-binding-target', 'avg-sim-target','n_neighbors-target','page_rank-target','0-target','1-target','2-target','3-target','4-target']

target_gene_names.head()

In [ ]:
#drug_pubchemIDs.columns = ['avg-binding-drug', 'avg-sim-drug','n_neighbors-drug','page_rank-drug','0-drug','1-drug','2-drug','3-drug','4-drug']
drug_pubchemIDs.head()

### 2.4 Building Train, Validation and Test Dataset using extracted features

In [ ]:

temp1 = pd.concat([target_gene_names]*68)
X = pd.concat([drug_target_binding.reset_index(drop=True), temp1.reset_index(drop=True)], axis=1)
X = X.sort_values(by=['Target', 'Drug'])
temp2 = pd.concat([drug_pubchemIDs]*442)
X = pd.concat([X.reset_index(drop=True), temp2.reset_index(drop=True)], axis=1)
X.index = drug_target_binding.index

##balancing data for further steps
repeatsize = np.size(np.where(X['Binding_Val'] <= 7))//np.size(np.where(X['Binding_Val'] > 7))
temp = pd.concat([X.iloc[np.where(X['Binding_Val'] > 7)]] * (repeatsize-1))
X = pd.concat([X, temp], axis=0, ignore_index=True)
##

X = X.sort_values(by=['Drug','Target'])


fX = X

Y = X.loc[:, 'Binding_Val']
X = X.loc[:, 'avg-binding-target':'4-drug']

split1 = train_test_split(X, test_size = 0.3)
X_train = split1[0]
split2 = train_test_split(split1[1], test_size = 0.3)
X_val = split2[0]
X_test = split2[1]

split1 = train_test_split(Y, test_size = 0.3)
Y_train = split1[0]
split2 = train_test_split(split1[1], test_size = 0.3)
Y_val = split2[0]
Y_test = split2[1]

X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape

## 3.XGboost 

To predict the continuous binding affinity for drug–target pairs, we will use [XGBoost](https://xgboost.readthedocs.io/en/latest/) library. 

I WILL tune following hyperparameters:
- `learning_rate`: Boosting learning rate
- `n_estimators`: Number of gradient boosted trees.
- `max_depth `: Maximum tree depth for base learners.
- `colsample_bytree`: Subsample ratio of columns when constructing each tree.
- `subsample`: Subsample ratio of the training instance.




Note: `drug_sim_threshold`, `target_sim_threshold` and `latent_dim` in Feature Engineering part can be viewed as hyperparameters too.

In [ ]:
import xgboost
from sklearn.metrics import mean_squared_error

### 3.1 Tune Hyperparameters

In [ ]:
def plot_model_results(results):
    epochs = len(results['validation_0']['rmse'])
    x_axis = range(0, epochs)
    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
    ax.plot(x_axis, results['validation_1']['rmse'], label='Validation')
    ax.legend()
    plt.ylabel('RMSE')
    plt.show()

In [ ]:

learning_rate = 0.1
n_estimators = 1000
max_depth = 7
colsample_bytree = 0.7
subsample = 0.7

model = xgboost.XGBRegressor(objective ='reg:linear', learning_rate = learning_rate, 
                             colsample_bytree = colsample_bytree,
                             max_depth = max_depth, 
                             subsample = subsample, 
                             n_estimators = n_estimators,
                             eval_metric='rmse')

model.fit(X_train,Y_train, eval_metric="rmse", 
          eval_set=[(X_train, Y_train), (X_val, Y_val)],
          verbose=False)


validation_rmse = np.sqrt(mean_squared_error(Y_val, model.predict(X_val)))

print("Validation RMSE: %.3f" % validation_rmse)
plot_model_results(model.evals_result())

### 3.2 Ploting Feature importance

In [ ]:
xgboost.plot_importance(model);

### 3.2 Evaluation

Let's make our perdiction binary. either a drug is binded to target or not, I will use $pK_d$ > 7 threshold as binded (drug-target) pair.

In [ ]:
from sklearn.metrics import confusion_matrix

test_rmse = np.sqrt(mean_squared_error(Y_test, model.predict(X_test)))



testpred = np.where(model.predict(X_test) > 7, 1, 0)
testreal = np.where(Y_test > 7, 1, 0)

conf = confusion_matrix(testreal, testpred)

test_acc = (conf[0,0] + conf[1,1])/testreal.shape[0]
test_percision = conf[1,1]/(conf[0,1] + conf[1,1])
test_recall = conf[1,1]/(conf[1,0] + conf[1,1])
test_f1 = 2*(test_percision*test_recall)/(test_percision + test_recall)


print("Test RMSE: %.3f" % test_rmse)
print("Test Accuracy: %.3f" % test_acc)
print("Test Percision: %.3f" % test_percision)
print("Test Recall: %.3f" % test_recall)
print("Test F1-Score: %.3f" % test_f1)
conf

## 4.Classification

I will change Binding values to binary values with threshold $pK_d$ > 7 
and use X_train to train



In [ ]:
Y_train = np.where(Y_train > 7, 1, 0)
Y_test = np.where(Y_test > 7, 1, 0)
Y_val = np.where(Y_val > 7, 1, 0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
modelrf = RandomForestClassifier()
modelrf.fit(X_train, Y_train)
conf = confusion_matrix(Y_test, modelrf.predict(X_test))

test_acc = (conf[0,0] + conf[1,1])/testreal.shape[0]
test_percision = conf[1,1]/(conf[0,1] + conf[1,1])
test_recall = conf[1,1]/(conf[1,0] + conf[1,1])
test_f1 = 2*(test_percision*test_recall)/(test_percision + test_recall)

print("Test Accuracy: %.3f" % test_acc)
print("Test Percision: %.3f" % test_percision)
print("Test Recall: %.3f" % test_recall)
print("Test F1-Score: %.3f" % test_f1)